Importing All the necessary Libraries

In [1]:
import os
import pandas as pd
import requests
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import re
import hyphenate


create stop words list and positive and negative word dictionary

In [2]:
stopwords_folder = "D:\\Beep Beep Boop Boop\\Python\\BlackCoffer Internship Assignment\\20211030 Test Assignment\\StopWords"
all_stopwords_list = []

for file_name in os.listdir(stopwords_folder):
    with open(os.path.join(stopwords_folder, file_name), 'r') as file:
        all_stopwords_list.extend(file.read().splitlines())
# all_stopwords_list=set(all_stopwords_list)


In [3]:
positive_words_path = "D:\\Beep Beep Boop Boop\\Python\\BlackCoffer Internship Assignment\\20211030 Test Assignment\\MasterDictionary\\negative-words.txt"
negative_words_path = "D:\\Beep Beep Boop Boop\\Python\\BlackCoffer Internship Assignment\\20211030 Test Assignment\\MasterDictionary\\positive-words.txt"

with open(positive_words_path, 'r') as positive_file:
    positive_words = positive_file.read().splitlines()

with open(negative_words_path, 'r') as negative_file:
    negative_words = negative_file.read().splitlines()

Add the link which you want to extract the text from

In [4]:
response = requests.get("https://insights.blackcoffer.com/database-discovery-tool-using-openai/")

Extract the article text and the title using beautiful soup html parser

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')
# Extract the title of the article from the <h1> tag
title_tag = soup.find('h1')
title = title_tag.text.strip() if title_tag else "Untitled"

potential_class_ids = ['td-post-content tagdiv-type', 'tdb-block-inner td-fix-index']

for class_id in potential_class_ids:
    text_elements = soup.find_all(class_=class_id)
    if text_elements:
        break

# Extract text from each element
text = []

for element in text_elements:
    # Remove '\n' tags
    element_text = element.get_text(separator='\n', strip=True)

    # Remove content within <pre> tag
    pre_tag = element.find('pre')
    if pre_tag:
        element_text = element_text.replace(pre_tag.get_text(strip=True), '')

    text.append(element_text)

# Join the text from all elements into a single string
final_text = '\n'.join(text)

# Include the title along with the content
final_content = f"Title: {title}\n\n{final_text}"
final_content = final_content.replace('\n',' ') #to remove all \n characters that get parsed with the article paragraphs
# print(final_content)


In [6]:
print(final_content)

Title: Database Discovery Tool using OpenAI  Client Background Client: A leading retail firm in the USA Industry Type: Retail Products & Services: Retail Business, e-commerce Organization Size: 100+ Problem Statement: Organizations often face challenges in managing and understanding their vast and complex databases. As data infrastructure evolves, new databases are introduced, and existing ones are modified, leading to a lack of comprehensive visibility into the entire data landscape. This lack of awareness poses several issues, including increased difficulty in ensuring data quality, security vulnerabilities, and inefficiencies in database administration. To address these challenges, there is a need for a Database Discovery Tool using OpenAI, aimed at providing an automated and intelligent solution for discovering, cataloging, and understanding the various databases within an organization’s ecosystem. Key Problems to Solve: Database Proliferation: Challenge: The rapid growth of databa

In [7]:
all_words = word_tokenize(final_content) #includes stopwords and punctuations

Before Sentiment Analysis is Performed, We need to remove the stopwords using the list of stop words provided.

In [8]:
tokens = word_tokenize(final_content)
filtered_tokens = [word for word in tokens if word.lower() not in all_stopwords_list]
# Join the tokens back into a single string
processed_text = ' '.join(filtered_tokens)

In [9]:
def remove_punctuation(text):
    no_punct = text.translate(str.maketrans('', '', string.punctuation))
    return no_punct

In [10]:
clean_text = remove_punctuation(processed_text)

In [11]:
tokens = word_tokenize(clean_text.lower())  # Convert to lowercase for case-insensitivity

In [12]:
total_clean_words = len(tokens)
print(total_clean_words)

447


In [13]:
# Calculate Positive Score
positive_score = sum(1 for word in tokens if word in positive_words)

# Calculate Negative Score (multiply by -1)
negative_score = -sum(-1 for word in tokens if word in negative_words)

print(positive_score)
print(negative_score)


22
26


In [14]:
#Polarity Score
polarity_score = (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)
print(polarity_score)

-0.08333333159722227


In [15]:
subjectivity_score = (positive_score + negative_score)/((total_clean_words)+ 0.000001)
print("Subjectivity Score: ",subjectivity_score)

Subjectivity Score:  0.10738255009534105


Analysis of Readability

In [16]:
number_of_words = len(word_tokenize(final_content)) - 2 #-2 to remove the count of Title and the ':' and for a better understanding of sentences, We take count of all words including stop words
number_of_sentences = len(sent_tokenize(final_content))

In [17]:
print(number_of_words)

795


In [18]:
Average_sentence_length = number_of_words/number_of_sentences
print(Average_sentence_length)

18.928571428571427


In [19]:
def count_syllables(word):
  """Counts the number of syllables in a word."""
  return len(hyphenate.hyphenate_word(word))

In [20]:
syllable_threshold = 2
number_of_complex_words = sum(1 for word in all_words if count_syllables(word) > syllable_threshold)
percentage_of_complex_words = number_of_complex_words/number_of_words
print(percentage_of_complex_words)

0.23647798742138365


In [21]:
fog_index  = 0.4 * (Average_sentence_length + percentage_of_complex_words)

In [22]:
average_number_of_words_per_sentence = number_of_words/number_of_sentences

In [23]:
complex_word_count = number_of_complex_words

For stopword removal using the stop word package of NLTK library

In [24]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [25]:
no_stop_words = remove_stopwords(final_content)

In [26]:
no_punctuation_and_stop_words = remove_punctuation(no_stop_words)

In [27]:
#for word count first tokenize the filtered text
tokens = word_tokenize(no_punctuation_and_stop_words)
word_count = len(tokens)
print(word_count)

476


In [28]:
def calculate_average_syllables_per_word(article_text):
    # Tokenize the article text into words
    no_pnct = remove_punctuation(article_text)
    words = word_tokenize(no_pnct)

    # Count syllables for each word
    syllable_counts = [count_syllables(word) for word in words]

    # Calculate total syllable count and total word count
    total_syllables = sum(syllable_counts)
    total_words = len(words)

    # Calculate average syllable count per word
    average_syllables_per_word = total_syllables / total_words

    return average_syllables_per_word

In [29]:
average_syllables_per_word  = calculate_average_syllables_per_word(final_content)

In [30]:
import re
def count_personal_pronouns(text):
    pronounRegex = re.compile(r'\b(I|we|my|our|you|your|they|them|ours|(?-i:us))\b',re.I)
    countof_pronouns = len(pronounRegex.findall(text))
    return countof_pronouns


In [31]:
count_of_personal_pronouns = count_personal_pronouns(final_content)

In [32]:
def calculate_average_word_length(text):
    words = text.split()
    total_characters = sum(len(word) for word in words)
    total_words = len(words)

    if total_words == 0:
        return 0  # Avoid division by zero

    avg_word_length = total_characters / total_words
    return avg_word_length


In [33]:
average_word_length = calculate_average_word_length(final_content)